In [1]:
#from Bio import AlignIO
from Bio.Align import substitution_matrices
#from Bio.Seq import Seq
#from Bio.SeqRecord import SeqRecord
#from Bio.Align import MultipleSeqAlignment

import numpy as np
import pandas as pd

# MSA filtering

**How does Yuri filter MSAs?**

The main filtering done in his scripts/phylogenetic reconstruction is to:
1) remove gaps
2) remove low "homogeneity" positions
3) intelligently add a consensus sequence

All of this involves sequence weighting. Sequence weighting is typically performed in MSAs to downweight highly redundant sequences and upweight diverse sequences. There are tree-based and distance-based weighting schemes. According to Henikoff & Henikoff 1994 [paper](https://doi.org/10.1016/0022-2836(94)90032-9), for both schemes, 
>"the weight assigned to a sequence is a measure of the distance between the sequence and a root or generalized sequence. Each distance is based on the entire sequence in question. However, the seq weight are typically applied to PSSMs in which each position is considered independently".

Both `sr_filter` and PSI-BLAST use Henikoff & Henikoff 1994 (HH94). According to this recent [paper](https://doi.org/10.1186/s12859-021-04183-8), their summary of the method is the following:
>the score of a sequence is the average of the scores of each position of the sequence, the score of a position being 1/rd, with r the number of different characters at the considered alignment column and d the number of times the character of the considered sequence and position appears in the considered alignment column. The idea of this weighting scheme is to give equal weight to all characters observed at one alignment column, dividing this weight equally among those sequences sharing that character at that position.

What they fail to mention from the Henikoff paper is the part about the consensus:
>"for every position of the alignment, the PSSM entry for each residue was the sequence- weighted observed frequency of that residue divided by its expected frequency tabulated from SWISS-PROT".

I asked Yuri how he filters MSAs. He told me to look at methods section 2.3 of a [paper of his](https://doi.org/10.1093/ve/veab015) and replied:
>Basically, the "best" amino acid is the one with the highest score (sr_filter has BLOSUM62 built-in) against the alignment column, calculated with sequence weights. Homogeneity prorates this score between the maximum (strictly homogeneous) and the non-pathological minimum (random assortment of amino acids). Consensus amino acid is registered when the homogeneity is above the threshold, otherwise it's set to "X".

In summary
1. Calculate HH94 score of a seq in an MSA
2. Calculate the Q score of an AA (x) in an MSA column as Qx = (HH94 score) * (blosum62 score for X against all AAs)
     - For each position in the MSA, an aligned AA is given a frequency score that is incremented by the HH94 score of the sequence. If the position is a gap, all AAs are possible, so each AA frequency is incremented by their default frequency in swiss prot*HH94 score. What we are left with is a vector of 20 amino acids and their frequencies for each column, where the frequencies of aligned AAs are typically much higher than their background swiss prot frequency.
3. The consensus AA for that position is the AA with the highest Qx score
4. Calculate the expectation of the score of an MSA column against a randomly selected AA (R) as Qr = (vector of relative frequencies of AAs) * Qscore
5. Calculate the homogeneity of an MSA column as H = (consensus - expectation) / (blosum score - expectation)
6. Keep/discar consensus AA depending on threshold 




Pseudocode
```
#get HH94 weights of sequences
for seq in MSA:
    calculate hh94 score
    
#get an expectation of AA substitutions
for AA1 (n=20):
 for AA2 (n=20):
    AAexpected = swiss_prot_frequency * blosum62(AA1xAA2)

#column specific frequencies
for col in msa:
    if AA = "-":
        ngaps += HH94_sequence_weight
    for seq in msa:
        if aligned_AA:
            AAfreq += HH94_sequence_weight
        elif:
            AAfreq += swiss_prot_frequency x HH94_sequence_weight
            
        #consensus
        for AA1(n=20):
            for AA2(n=20):
                AA1_weight += blosum62(AA1xAA2) * AAfreq(AA2)
                
        bestAA = argmax(AA1_weights); AA1_weights > 0
        
        #homogeneity
        if bestAA/nseqs > AAexpected:
            homogeneity = (bestAA / nseqs - bestAAexpected) / (blosum62(bestAAxbestAA) - bestAAexpected)
            
        if ngaps <= nseq*grcon:
            if homogeneity > homo_cutoff
                consensusAA = bestAA
         else:
             consensusAA = X
            
                
```
        
    

# Functions

### background tables

In [2]:
def get_tables_for_msa_filtering():
    
    #set a background frequency of AAs
    background_aa_freq = {
                            "A" : 0.07422,
                            "C" : 0.02469,
                            "D" : 0.05363,
                            "E" : 0.05431,
                            "F" : 0.04742,
                            "G" : 0.07415,
                            "H" : 0.02621,
                            "I" : 0.06792,
                            "K" : 0.05816,
                            "L" : 0.09891,
                            "M" : 0.02499,
                            "N" : 0.04465,
                            "P" : 0.03854,
                            "Q" : 0.03426,
                            "R" : 0.05161,
                            "S" : 0.05723,
                            "T" : 0.05089,
                            "V" : 0.07292,
                            "W" : 0.01303,
                            "Y" : 0.03228,
                         }
        
    #get a blosum62 matrix
    blosum62 = substitution_matrices.load("BLOSUM62")

    #set a expectation of AAs
    #expectation_aa = {}
    #for aa1 in background_aa_freq.keys():
    #    expectation_aa[aa1] = 0
    #    for aa2 in background_aa_freq.keys():
    #        expectation_aa[aa1] = expectation_aa[aa1] + blosum62[aa1,aa2]*background_aa_freq[aa2]
    
    expectation_aa = aa_freq_to_score(background_aa_freq)

    return background_aa_freq, blosum62, expectation_aa

### AA frequency to score

In [3]:
def aa_freq_to_score(col_aa_frq):
    """
    expects a dictionary of AA : frequency
    returns a dictionary of AA : score
    
    score is sum of blosum62 scores of an index AA versus all other AAs
    """
    #set all AA weights as zero
    weights = {}
    for aa in col_aa_frq.keys():
        weights[aa] = 0
        
    #get a blosum62 matrix
    blosum62 = substitution_matrices.load("BLOSUM62")
    
    #assign weights to each AA
    for aa1 in col_aa_frq.keys():
        for aa2 in col_aa_frq.keys():
            weights[aa1] = weights[aa1] + (blosum62[aa1,aa2] * col_aa_frq[aa2])
            #print(f"AA1 is {aa1}, AA2 is {aa2}, frequency of AA2 is {col_aa_frq[aa2]}, blosum62 of {aa1}{aa2} is {blosum62[aa1,aa2]}, updating weight of AA1 to {weights[aa1]}")
    
    return weights

### msa_to_df

In [4]:
#deprecated
def aln_to_df(aln, msa_format='fasta'):
    """
    convertsa  biopython alignment object into a pandas dataframe
    """
    
    headers = [rec.id for rec in aln]
    seqs = [list(rec) for rec in aln]
    df = pd.DataFrame.from_records(seqs, index = headers).astype('str')
    return df

In [5]:
def fa_to_df(fa, is_msa=False):
    """
    Converts a fasta-formatted file to a pandas dataframe
    """
    with open(fa) as infile:
        headers = []
        seqs = []
        for line in infile:
            if line.startswith(">"):
                header = line.strip(">").split()[0]
                headers.append(header)
            else:
                seq = list(line.strip())
                seqs.append(seq)
        
    df = pd.DataFrame.from_records(seqs, index = headers)
    
    #check to make sure the MSA is aligned properly
    if is_msa:
        assert df.isna().sum().sum() == 0, f"All sequences must be the same length"
   
    return df

def df_to_fa(df, output):
    """
    converts a pandas dataframe of sequences to a fasta-formatted file
    """
    
    #combine the columns into a single string
    series = df.T.apply("".join)
    
    #write the output
    with open(output, 'w') as o:
        for header, seq in series.items():
            print(f">{header}", seq, sep = '\n', file = o)

### get_gap_cols_from_msa

In [96]:
def get_gap_cols_from_msa(df, grcut=1):
    """
    -grcut = no more than r fraction of gaps
    
    returns a list of columns that are more than r fraction of gaps
    """
    #df = msa_to_df(msa)
    num_seqs, num_columns = df.shape
    EPSILON = 1e-6

    #calculate the minimum number of non-NA values per column
    max_gaps_in_column = round((num_seqs * grcut) + EPSILON, 3)
    

    #convert gaps to NA, count how many NAs per column
    gap_count_series = df.replace("-", np.nan).isnull().sum()

    #get columns where number of NAs is > max_gaps
    gap_col_list = gap_count_series.loc[lambda s: s > max_gaps_in_column].index.tolist()
    #df2 = df.loc[:, good_cols]
    
    ##RO
    
    return gap_col_list

### calc_HH94

In [8]:
def calc_HH94(df, grswe=0.51):
    """
    - sequences have an initial weight of 1/nseq
    - Calculates the score of a position in an MSA as 1/rd
        r = # of dif characters (excluding gaps) in a column
        d = number of times the character of the considered sequence 
        and position appears in the considered alignment column
    
    - columns in MSA that are >grswe fraction of gap positions are ignored
    
    - the scores of a sequence's aligned positions are added to its weight
    
    Returns a series object containing the sequence name and HH94 weight
    """
    #df = msa_to_df(msa)
    
    #convert gaps to NA so they are not counted in the weights 
    df2 = df.replace("-", np.nan)
    
    #get a list of gap columns that are more than grswe fraction of gaps
    gap_cols = get_gap_cols_from_msa(df, grswe)
    
    #initial sequence weight is just 1 / nseq
    initial_weight = 1 / len(df)

    
    colaa_weights_nest_dict = {}
    for column in df2.columns:
        if column not in gap_cols:
        
            #get a count of the frequency of each AA in the column
            #by default, NAs (gaps) are not counted
            col_series = df2[[column]].value_counts()

            #calculate the AA weights in the column
            #the length of the series is the number of unique AAs
            colaa_weights = round(1 / (col_series * len(col_series)), 3)

            #store the column-specific results in a dict of dict
            colaa_weights_dict = colaa_weights.to_dict()
            colaa_weights_nest_dict[column] = colaa_weights_dict

    #replace the AAs in the MSA with their column-specific weights
    df3 = df2.replace(colaa_weights_nest_dict)
    
    #for each sequence, sum all the values and add the initial wieght
    hh94_series = df3.sum(axis = 1).round(3) + initial_weight
    
    #normalize
    hh94_mean = hh94_series.mean()
    if hh94_mean < 0:
        hh94_mean = 1
    hh94_norm = round(hh94_series / hh94_series.mean(), 4)
    
    #return a dictionary containing the sequence and its normalized weight
    return hh94_norm.to_dict()
    

### col AA frequencies

In [9]:
def get_col_aa_frequencies(column, hh94_scores, background_aa_freq):
    """
    Accepts a dictionary or pandas series containing seq : aligned char
    
    calculates the column=specific AA weights, given HH94 scores of sequences
    
    returns a dictionary of AA : weight
    """
    
    #set all AA frequencies as zero
    aafreq = {}
    for aa in background_aa_freq.keys():
        aafreq[aa] = 0
    
    #set gap score as zero
    ngap = 0
    
    for seq,char in column.items():
        if char == "-":
            
            #increment the background frequencies of all AAs
            for aa, score in aafreq.items():
                aafreq[aa] = aafreq[aa] + background_aa_freq[aa]*hh94_scores[seq]
            
            #increment the gap frequency 
            ngap += hh94_scores[seq]
        else:
            
            aafreq[char] = aafreq[char] + hh94_scores[seq]
    
    #add in ngaps to the dict
    aafreq["-"] = ngap
    
    return aafreq

### best AA

In [10]:
def get_best_aa_and_score(aascores):
    """
    Expects a dictionary of AA : score
    Selects the AA with the highest score
    
    Returns AA and score
    """
    
    #get top AA and score, as a tuple
    bestaa_tuple = max(aascores.items(), key = lambda k : k[1])
    
    return bestaa_tuple[0],bestaa_tuple[1]
    
    

### homogeneity

In [11]:
def calc_homogeneity(aa, score, nseqs, expectaascores, blosum62):
    """
    calculates the homogeneity of a column in an MSA
    homogeneity ranges from 0-1
    """
    
    homogeneity = (score/nseqs - expectaascores[aa]) / (blosum62[aa, aa] - expectaascores[aa])
    return round(homogeneity, 4)

### Consensus

In [15]:
def assign_consensus(bestaa, grcon, homo, hocon, ngaps, nseqs, ambiguousAA="X"):
    """
    
    """
    #Default consensus is gap
    caa = "-"

    if ngaps <= nseqs * grcon:
        if homo > hocon:
            caa = bestaa
        else:
            caa = ambiguousAA

    return caa


# manual pipeline

In [83]:
background_aa_freq,blosum62,expectedAAscores = get_tables_for_msa_filtering()

In [309]:
df = fa_to_df('tmp3.afa', is_msa=True)
df

,0,1,2,3,4,5
seq1,M,S,E,D,S,I
seq2,M,S,E,K,T,I
seq3,-,M,E,D,K,I
seq4,-,-,E,D,-,-


In [85]:
hh94_dict = calc_HH94(df)
hh94_dict

{'seq1': 1.1905, 'seq2': 1.3808, 'seq3': 1.0476, 'seq4': 0.3812}

In [ ]:
column = df[0]

col_aa_frq = get_col_aa_frequencies(column, hh94_dict, background_aa_freq)
col_aa_frq
ngaps = col_aa_frq.popitem()[1]

In [ ]:
col_aa_scores = aa_freq_to_score(col_aa_frq)
bestaa,bestaascore = get_best_aa_and_score(col_aa_scores)
print(bestaa)
print(bestaascore)
#col_aa_scores

#weights

In [ ]:
nseqs = 4
print(f"bestwt is {bestaascore}, nseqs is {nseqs}, pse is 0, bestAA is {bestaa}, bestAAexpect is {expectedAAscores[bestaa]}, blosumscore is {blosum62[bestaa,bestaa]}")

In [ ]:
if bestaascore/nseqs > expectedAAscores[bestaa]:
    hom = calc_homogeneity(bestaa, bestaascore, nseqs, expectedAAscores, blosum62)
print(hom)

In [ ]:

grcon = 0.499
hocon = 0.33
con = assign_consensus(bestaa, grcon, hom, hocon, ngaps)
print(ngaps)
print(con)

# Wrapper

In [97]:
def filter_msa(msa, msa_out, grcut=1, hocut=-100, gcon=0.499, hcon=0.333, conplus=False, grswe=0.51):
    """
    MSA filtering + trimming, as described in 
    Esterman et. al (2021) https://doi.org/10.1093/ve/veab015
    """

    bad_cols = []
    
    if grcut < 1:
        print(f'getting a list of positions that are > {grcut * 100}% gaps')
        
        df = fa_to_df(msa, is_msa=True)
        
        #get a list of gap positions
        gapcols = get_gap_cols_from_msa(df, grcut)
        
        bad_cols += gapcols
        
        
    if hocut >= 0 or conplus:
        
        #then we need to iterate through each position in the MSA
        print(f'iterating through the alignment')
        
        #consensus seq is empty
        conseq = []
        
        #get the necessary tables
        background_aa_freq,blosum62,expectedAAscores = get_tables_for_msa_filtering()
        
        #convert the msa to a dataframe 
        df = fa_to_df(msa, is_msa=True)
        nseqs, ncols = df.shape
        
        #get the HH94 scores of the sequences
        hh94_scores = calc_HH94(df, grswe)
        
        for col in range(0,ncols):
            
            #get the column
            column = df[col]

            #calculate the AA frequencies
            col_aa_frq = get_col_aa_frequencies(column, hh94_scores, background_aa_freq)

            #the last item in the dictionary is the gap frequencies
            #record it and remove from the dictionary,
            #otherwise the next fxn throws an error
            ngaps = col_aa_frq.popitem()[1]

            #convert the frequencies to scores
            col_aa_scores = aa_freq_to_score(col_aa_frq)

            #get the top scoring AA
            bestaa, bestaascore = get_best_aa_and_score(col_aa_scores)
                
            #assess the homogeneity of the AA, if the bestscoring AA is above expectation
            if bestaascore/nseqs > expectedAAscores[bestaa]:
                homo = calc_homogeneity(bestaa, bestaascore, nseqs, expectedAAscores, blosum62)
            
            #record columns that are below the homogeneity threshold
            if homo < hocut:
                bad_cols.append(col)

            #assign the consensus
            con = assign_consensus(bestaa, gcon, homo, hcon, ngaps, nseqs)
            conseq.append(con)
        
        if conplus:
            
            #make a dataframe from the consensus
            df_c = pd.DataFrame.from_records([conseq], index = ["CONSENSUS"])
            
            #update the original MSA datafarme
            df = pd.concat([df_c,df])
        
        #remove gap/nonhomogenous columns
        if len(bad_cols) > 0:
            df = df.drop(columns = bad_cols)
            
        #write the output
        with open(msa_out, 'w') as f:
            df_to_fa(df, msa_out)
        
        return df
        
    else:
        print('doing nothing, printing alignment')
        df = fa_to_df(msa, is_msa=True)
        with open(msa_out, 'w') as f:
            df_to_fa(df, msa_out)
        

In [ ]:
%%time
conseq = filter_msa('C19_cas8b2.FASTA','test.afa', grcut=0.667, hcon=0.33, conplus=True, hocut=0.05)
conseq

# Notes

- I noticed my background expect AA frequencies differ from Yuris. It turns out, his blosum62 matrix is different than mine/ For example, Yuri scores RD as -1, but mine is -2. According to [BLAST]((https://ftp.ncbi.nlm.nih.gov/blast/matrices/BLOSUM62), it is -2 


In [ ]:
print(blosum62["R","D"])
print(blosum62["R","V"])
#print(blosum62)



I typically use `-hocut=0.1` for filtering the MSA prior to tree construction. 

Looking at `prof_align`, `-hocut` isn't invoked.

Looking at `cog_psicognitor`, which invokes `run_psiprofile`, the srfilter command is (If there is consensus, don't bother filtering):
>"sr_filter -conplus -gcon=.499 -hcon=0 -grcut=.499" unless(xcons)

So, hocut is invoked as part of the `-hcon` command, which asserts that the minimum homogeneity of the consensus must be 0.





So how does Yuri calculate homogeneity?
```
$hom = ($bestwt/($pcnt+$pse) - $expect{$bestaa})/($simmat{$bestaa.$bestaa} - $expect{$bestaa}) if($bestaa ne "" and $bestwt/($nseq+$pse)>$expect{$bestaa});
```

perform the homogoneity calculation if (bestaa does not equal "" and bestwt/nseq+pse > expect(bestaa))


1. bestwt = -100.0*($nseq+$pse) 
    -nseq = # of seqs in MSA
    -pse = pseudocounts, default 0
2. pcnt = # of seqs in MSA, adjusted if we exclude gaps (we don't, by default). So, basically just nseq.
3.
....?  I should just ask him.